In [1]:
# 导入必要的库
import cv2 as cv
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output
import matplotlib.pyplot as plt
from grayscale import *
from binarization import *
from sharpening import *
from filter import *
from geometric_transform import *
from region_growing import *
from edge_detection import *
from dft import *
from morphology_process import *
from geometry import *
from contour import *

In [2]:
# 图像显示函数
def show_image(image, title='Image'):
    plt.imshow(cv.cvtColor(image, cv.COLOR_BGR2RGB))
    plt.title(title)
    plt.axis('off')
    plt.show()

# 图像处理函数
def process_image(image, method, sub_option, kernel_size=None, sigmaX=None, d=None, sigmaColor=None, sigmaSpace=None, threshold=None):
    '''
    图像处理函数
    
    :param image: 输入图像
    :param method: 方法
    :param sub_option: 子选项
    :param kernel_size: 核大小
    :param sigmaX: 高斯核标准差
    :param d: 双边滤波参数
    :param sigmaColor: 双边滤波参数
    :param sigmaSpace: 双边滤波参数
    :param threshold: 阈值

    return: 处理后的图像
    '''

    gray_image = grayscale(image)
    processed_image = None
    
    if method == '灰度化':
        if sub_option == '灰度化':
            processed_image = gray_image
        elif sub_option == '灰度直方图修正':
            processed_image = histogram_equalization(gray_image)
        elif sub_option == '线性变换':
            processed_image = linear_transformation(gray_image)
        elif sub_option == '对数变换':
            processed_image = log_transformation(gray_image)
        elif sub_option == '指数变换':
            processed_image = exponential_transformation(gray_image)
        plot_histogram(processed_image) # 绘制直方图
    elif method == '二值化':
        if sub_option == '固定阈值分割':
            processed_image = fixed_threshold(gray_image, threshold)
        elif sub_option == '自适应均值阈值分割':
            processed_image = adaptive_mean_threshold(gray_image)
        elif sub_option == '自适应高斯阈值分割':
            processed_image = adaptive_gaussian_threshold(gray_image)
        elif sub_option == 'Otsu自动计算阈值分割':
            processed_image = otsu_threshold(gray_image)
        
    elif method == '滤波':
        if sub_option == '均值滤波':
            kernel_size = kernel_size or 5
            processed_image = cv.blur(gray_image, (kernel_size, kernel_size))
        elif sub_option == '高斯滤波':
            kernel_size = kernel_size or 5
            sigmaX = sigmaX or 1.5
            processed_image = cv.GaussianBlur(gray_image, (kernel_size, kernel_size), sigmaX)
        elif sub_option == '中值滤波':
            kernel_size = kernel_size or 5
            processed_image = cv.medianBlur(gray_image, kernel_size)
        elif sub_option == '双边滤波':
            d = d or 9
            sigmaColor = sigmaColor or 75
            sigmaSpace = sigmaSpace or 75
            processed_image = cv.bilateralFilter(gray_image, d, sigmaColor, sigmaSpace)

    elif method == '锐化': 
        if sub_option == 'Sobel算子':
            processed_image = sobel_sharpen(image)
        elif sub_option == 'Laplacian算子':
            processed_image = laplacian_sharpen(image)

    elif method == '图像分割':
        processed_image = region_growing(image, seed_point=(400, 100) , threshold=8)

    elif method == '频域滤波':
        processed_image = frequency_filter(image, sub_option)

    elif method == '形态学处理':
        processed_image = morphology_process(image, sub_option)

    elif method == '几何形状检测':
        processed_image = geometric_dectect(image, sub_option)

    elif method == '轮廓提取':
        processed_image = contours(image, sub_option, threshold)
        
        
    
    return processed_image

In [ ]:
# 文件上传控件
file_upload = widgets.FileUpload(
    accept='image/*',
    multiple=False
)

# 方法选择下拉菜单
method_dropdown = widgets.Dropdown(
    options=['灰度化', '二值化', '滤波', '锐化', '几何变换', '图像分割', 
             '边缘检测', '频域滤波', '形态学处理', '几何形状检测', '轮廓提取'],  
    value='灰度化',
    description='处理方法:'
)

# 子选项选择下拉菜单
sub_option_dropdown = widgets.Dropdown(
    options=['灰度化', '灰度直方图修正', '线性变换', '对数变换', '指数变换'],
    value='灰度化',
    description='子选项:'
)

# 处理按钮
process_button = widgets.Button(
    description='处理',
)

# 输出区域
output = widgets.Output()


# 文件上传事件处理
def on_file_upload_changed(change):
    global image
    uploaded_files = change['new']
    if isinstance(uploaded_files, tuple):
        uploaded_files = {file.name: {'content': file.content} for file in uploaded_files}
    
    for filename, file_info in uploaded_files.items():
        content = file_info['content']
        image = cv.imdecode(np.frombuffer(content, np.uint8), -1)
        with output:
            clear_output(wait=True)
            show_image(image, '原始图像')
file_upload.observe(on_file_upload_changed, names='value')

# 处理按钮点击事件处理
def on_process_button_clicked(b):
    with output:
        clear_output(wait=True)
        if method_dropdown.value == '二值化' and sub_option_dropdown.value == '固定阈值分割':
            processed_image = process_image(image, method_dropdown.value, sub_option_dropdown.value, threshold=threshold_slider.value)
        
        elif method_dropdown.value == '滤波':
            kernel_size = filter_params.children[0].value
            sigmaX = filter_params.children[1].value
            d = filter_params.children[2].value
            sigmaColor = filter_params.children[3].value
            sigmaSpace = filter_params.children[4].value
            processed_image = process_image(image, method_dropdown.value, sub_option_dropdown.value, kernel_size, sigmaX, d, sigmaColor, sigmaSpace)
        
        elif method_dropdown.value == '几何变换':
            if sub_option_dropdown.value == '平移':
                tx = geometry_params.children[0].value
                ty = geometry_params.children[1].value
                processed_image = translate(image, tx, ty)
            elif sub_option_dropdown.value == '旋转':
                angle = geometry_params.children[2].value
                processed_image = rotate(image, angle)
            elif sub_option_dropdown.value == '错切':
                processed_image = shear(image)
            elif sub_option_dropdown.value == '缩放':
                scale_factor = geometry_params.children[3].value
                processed_image = scale(image, scale_factor)
            elif sub_option_dropdown.value == '翻转':
                flip_direction = geometry_params.children[4].value
                processed_image = flip(image, flip_direction)
        
        elif method_dropdown.value == '边缘检测':
            edge_method = edge_detection_params.children[0].value
            if edge_method == 'Sobel算子':
                processed_image = sobel_edge_detection(image)  
            elif edge_method == 'Laplacian算子':
                processed_image = laplacian_edge_detection(image)
            elif edge_method == 'Canny算子':
                low_threshold = edge_detection_params.children[1].value
                high_threshold = edge_detection_params.children[2].value
                processed_image = canny_edge_detection(image, low_threshold, high_threshold)     
        elif method_dropdown.value == '轮廓提取':
            if sub_option_dropdown.value == '轮廓绘制':
                processed_image = process_image(image, method_dropdown.value, sub_option_dropdown.value, threshold=threshold_slider.value)
        else:
            processed_image = process_image(image, method_dropdown.value, sub_option_dropdown.value)
        
        fig, axs = plt.subplots(1, 2, figsize=(12, 6))
        axs[0].imshow(cv.cvtColor(image, cv.COLOR_BGR2RGB))
        axs[0].set_title('原始图像')
        axs[0].axis('off')
        
        axs[1].imshow(cv.cvtColor(processed_image, cv.COLOR_BGR2RGB) if len(processed_image.shape) == 3 else processed_image, cmap='gray')
        axs[1].set_title('处理后的图像')
        axs[1].axis('off')
        
        plt.show()
process_button.on_click(on_process_button_clicked)

# 方法改变事件处理
def on_method_change(change):
    threshold_slider.layout.visibility = 'hidden'
    filter_params.layout.visibility = 'hidden'
    geometry_params.layout.visibility = 'hidden'
    edge_detection_params.layout.visibility = 'hidden'

    if change['new'] == '灰度化':
        sub_option_dropdown.options = ['灰度化', '灰度直方图修正', '线性变换', '对数变换', '指数变换']

    elif change['new'] == '二值化':
        sub_option_dropdown.options = ['固定阈值分割', '自适应均值阈值分割', '自适应高斯阈值分割', 'Otsu自动计算阈值分割']
        if sub_option_dropdown.value == '固定阈值分割':
            threshold_slider.layout.visibility = 'visible'
        else:
            threshold_slider.layout.visibility = 'hidden'

    elif change['new'] == '滤波':
        sub_option_dropdown.options = ['均值滤波', '高斯滤波', '中值滤波', '双边滤波']
        filter_params.layout.visibility = 'visible'

        if sub_option_dropdown.value in ['均值滤波', '高斯滤波']:
            filter_params.children[0].layout.visibility = 'visible'  # Kernel Size
            filter_params.children[1].layout.visibility = 'visible' if sub_option_dropdown.value == '高斯滤波' else 'hidden'  # Sigma X
            filter_params.children[2].layout.visibility = 'hidden'  # Diameter
            filter_params.children[3].layout.visibility = 'hidden'  # Sigma Color
            filter_params.children[4].layout.visibility = 'hidden'  # Sigma Space
        elif sub_option_dropdown.value == '双边滤波':
            filter_params.children[0].layout.visibility = 'hidden'  # Kernel Size
            filter_params.children[1].layout.visibility = 'hidden'  # Sigma X
            filter_params.children[2].layout.visibility = 'visible'  # Diameter
            filter_params.children[3].layout.visibility = 'visible'  # Sigma Color
            filter_params.children[4].layout.visibility = 'visible'  # Sigma Space
        else:
            filter_params.children[0].layout.visibility = 'visible'  # Kernel Size
            filter_params.children[1].layout.visibility = 'hidden'  # Sigma X
            filter_params.children[2].layout.visibility = 'hidden'  # Diameter
            filter_params.children[3].layout.visibility = 'hidden'  # Sigma Color
            filter_params.children[4].layout.visibility = 'hidden'  # Sigma Space
        
    elif change['new'] == '锐化':
        sub_option_dropdown.options = ['Sobel算子', 'Laplacian算子']

    elif change['new'] == '几何变换':
        sub_option_dropdown.options = ['平移', '旋转', '错切', '缩放', '翻转']
        geometry_params.layout.visibility = 'visible'

        # 初始化几何变换参数控件的可见性
        for param in geometry_params.children:
            param.layout.visibility = 'hidden'
        
        if sub_option_dropdown.value == '平移':
            geometry_params.children[0].layout.visibility = 'visible'  # Translation X
            geometry_params.children[1].layout.visibility = 'visible'  # Translation Y
        elif sub_option_dropdown.value == '旋转':
            geometry_params.children[2].layout.visibility = 'visible'  # Rotation Angle
        elif sub_option_dropdown.value == '缩放':
            geometry_params.children[3].layout.visibility = 'visible'  # Scale Factor
        elif sub_option_dropdown.value == '翻转':
            geometry_params.children[4].layout.visibility = 'visible'  # Flip Direction
    
    elif change['new'] == '图像分割':
        sub_option_dropdown.options = ['区域生长']

    elif change['new'] == '边缘检测':
        sub_option_dropdown.options = ['Sobel算子', 'Laplacian算子', 'Canny算子']
        edge_detection_params.layout.visibility = 'visible'

    elif change['new'] == '频域滤波':
        sub_option_dropdown.options = ['低通滤波', '高通滤波', '带阻(陷波)滤波']

    elif change['new'] == '形态学处理':
        sub_option_dropdown.options = ['膨胀', '腐蚀', '开运算', '闭运算', '形态学梯度', '顶帽', '黑帽']

    elif change['new'] == '几何形状检测':
        sub_option_dropdown.options = ['轮廓检测']
    
    elif change['new'] == '轮廓提取':
        sub_option_dropdown.options = ['轮廓绘制']
        if sub_option_dropdown.value == '轮廓绘制':
            threshold_slider.layout.visibility = 'visible'
        else:
            threshold_slider.layout.visibility = 'hidden'
        
method_dropdown.observe(on_method_change, names='value')

# 子选项改变事件处理
def on_sub_option_change(change):
    filter_params.layout.visibility = 'hidden'
    geometry_params.layout.visibility = 'hidden'
    threshold_slider.layout.visibility = 'hidden'
    edge_detection_params.layout.visibility = 'hidden'

    if method_dropdown.value == '二值化':
        if change['new'] == '固定阈值分割':
            threshold_slider.layout.visibility = 'visible'
        else:
            threshold_slider.layout.visibility = 'hidden'

    elif method_dropdown.value == '滤波':
        filter_params.layout.visibility = 'visible'
        if change['new'] in ['均值滤波', '高斯滤波']:
            filter_params.children[0].layout.visibility = 'visible'  # Kernel Size
            filter_params.children[1].layout.visibility = 'visible' if change['new'] == '高斯滤波' else 'hidden'  # Sigma X
            filter_params.children[2].layout.visibility = 'hidden'  # Diameter
            filter_params.children[3].layout.visibility = 'hidden'  # Sigma Color
            filter_params.children[4].layout.visibility = 'hidden'  # Sigma Space
        elif change['new'] == '双边滤波':
            filter_params.children[0].layout.visibility = 'hidden'  # Kernel Size
            filter_params.children[1].layout.visibility = 'hidden'  # Sigma X
            filter_params.children[2].layout.visibility = 'visible'  # Diameter
            filter_params.children[3].layout.visibility = 'visible'  # Sigma Color
            filter_params.children[4].layout.visibility = 'visible'  # Sigma Space
        else:
            filter_params.children[0].layout.visibility = 'visible'  # Kernel Size
            filter_params.children[1].layout.visibility = 'hidden'  # Sigma X
            filter_params.children[2].layout.visibility = 'hidden'  # Diameter
            filter_params.children[3].layout.visibility = 'hidden'  # Sigma Color
            filter_params.children[4].layout.visibility = 'hidden'  # Sigma Space

    elif method_dropdown.value == '几何变换':
        geometry_params.layout.visibility = 'visible'
        for param in geometry_params.children:
            param.layout.visibility = 'hidden'
        if change['new'] == '平移':
            geometry_params.children[0].layout.visibility = 'visible'  # Translation X
            geometry_params.children[1].layout.visibility = 'visible'  # Translation Y
        elif change['new'] == '旋转':
            geometry_params.children[2].layout.visibility = 'visible'  # Rotation Angle
        elif change['new'] == '缩放':
            geometry_params.children[3].layout.visibility = 'visible'  # Scale Factor
        elif change['new'] == '翻转':
            geometry_params.children[4].layout.visibility = 'visible'  # Flip Direction

    elif method_dropdown.value == '边缘检测':
        edge_detection_params.layout.visibility = 'visible'
        edge_detection_params.children[1].layout.visibility = 'visible' if change['new'] == 'Canny算子' else 'hidden'
        edge_detection_params.children[2].layout.visibility = 'visible' if change['new'] == 'Canny算子' else 'hidden'
    
    elif method_dropdown.value == '轮廓提取':
        if change['new'] == '轮廓绘制':
            threshold_slider.layout.visibility = 'visible'


sub_option_dropdown.observe(on_sub_option_change, names='value')

# 显示控件
display(file_upload, method_dropdown, sub_option_dropdown, threshold_slider, filter_params, geometry_params, edge_detection_params, process_button, output)

FileUpload(value=(), accept='image/*', description='Upload')

Dropdown(description='处理方法:', options=('灰度化', '二值化', '滤波', '锐化', '几何变换', '图像分割', '边缘检测', '频域滤波', '形态学处理', '几何形…

Dropdown(description='子选项:', options=('灰度化', '灰度直方图修正', '线性变换', '对数变换', '指数变换'), value='灰度化')

IntSlider(value=99, description='阈值:', layout=Layout(visibility='visible'), max=255)

Button(description='处理', style=ButtonStyle())

Output()